In [ ]:
!pip install keras_metrics owlready2 fasttext

from google.colab import drive
drive.mount("/content/drive")

import sys
sys.path.insert(0, '/content/drive/My Drive/Colab Notebooks')

     |████████████████████████████████| 20.8MB 1.4MB/s 
     |████████████████████████████████| 71kB 4.9MB/s 
  Created wheel for owlready2: filename=Owlready2-0.24-cp36-cp36m-linux_x86_64.whl size=20263010 sha256=3087832b7d7900cd895dda71bc0dfd43d40a0c6111a3d9bd1afa22d587932e57
  Stored in directory: /root/.cache/pip/wheels/66/87/0b/7a97bd19c9dc817b279aadb5dd920ce3a77050e2ab5a413f43
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3015422 sha256=f19fdd318a904c6235960414bf6a461506346dff9b56a9caf101f3cf076fc81d
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built owlready2 fasttext
Mounted at /content/drive


# Création fichier paires négatives


In [ ]:
def nettoyer_unicode(c):
    liste_codes =  {
        'Ã\xa0': 'à',
        'Ã€': 'À',
        'Ã¢': 'â',
        'ã': 'â',
        'Ã‚': 'Â',
        'Ã©': 'é',
        'Ã©': 'é',
        'ã‰': 'é',
        'ãƒâ©': 'é',
        'Ã\x89': 'É',
        'Ã\xa8': 'è',
        'Ã\xaa': 'ê',
        'Ã\x8a': 'Ê', 
        'Ã¨': 'ê',       
        'Ã«': 'ë',
        'Ã®':'î',
        'Ã\x8e':'Î',        
        'Ã¯': 'ï', 
        'Ã´': 'ô',
        'Ã\x94': 'Ô',
        'Ã¹': 'ù', 
        'Ã»': 'û',
        'Å\x93': 'œ', 
        'Â«': '«',
        'Â»': '»',
        'Ã§': 'ç',
        'Ã\x87': 'Ç',
        'Âº': 'º',
        'â€™': '’',
        'â\x80\x99': '’',
        'â\x80\xa6' : '…',
    }
    
    for code in liste_codes:
        c = c.replace(code,liste_codes[code])
    return c

In [ ]:
import numpy as np
from sklearn.utils import shuffle

#Paires positives
paires_pos = np.load("/content/drive/My Drive/Colab Notebooks/paires_pos.npy", allow_pickle = True)
# paires_pos = np.load("Desktop/paires_pos.npy", allow_pickle = True)

#Paires negatives
paires_neg = np.load("/content/drive/My Drive/Colab Notebooks/paires_neg.npy", allow_pickle = True)
# paires_neg = np.load("Desktop/paires_neg.npy", allow_pickle = True)

print(len(paires_pos), "paires positives")
print(len(paires_neg), "paires negatives")

#training
rand_pos_indices = np.random.choice(paires_pos.shape[0], 8250, replace=False)
training_data_pos = paires_pos[rand_pos_indices]
rand_neg_indices = np.random.choice(paires_neg.shape[0], 8250, replace=False)
training_data_neg = paires_neg[rand_neg_indices]

#training data
# formatter en X et Y pour CNN
id = []
titre1 = []
titre2 = []
for pair in training_data_neg:
  id.append(0)
  titre1.append(nettoyer_unicode(pair[0].lower()))
  titre2.append(nettoyer_unicode(pair[1].lower()))
for pair in training_data_pos:
  id.append(1)
  titre1.append(nettoyer_unicode(pair[0].lower()))
  titre2.append(nettoyer_unicode(pair[1].lower()))
train_data = shuffle(titre1, titre2, id)

#test
#get remaining pairs
remaining_pos_indices = np.array([x for x in np.arange(paires_pos.shape[0]) if not np.isin(x, rand_pos_indices)])
remaining_neg_indices = np.array([x for x in np.arange(paires_neg.shape[0]) if not np.isin(x, rand_neg_indices)])
remaining_pos = paires_pos[remaining_pos_indices]
remaining_neg = paires_neg[remaining_neg_indices]

#sélection aléatoire 500 paires positives et négatives
test_indices_pos = np.random.choice(remaining_pos.shape[0], 500, replace=False)
test_data_pos = remaining_pos[test_indices_pos]
test_indices_neg = np.random.choice(remaining_neg.shape[0], 500, replace=False)
test_data_neg = remaining_neg[test_indices_neg]

id = []
titre1 = []
titre2 = []
for pair in test_data_neg:
  id.append(0)
  titre1.append(nettoyer_unicode(pair[0].lower()))
  titre2.append(nettoyer_unicode(pair[1].lower()))
for pair in test_data_pos:
  id.append(1)
  titre1.append(nettoyer_unicode(pair[0].lower()))
  titre2.append(nettoyer_unicode(pair[1].lower()))
test_data = shuffle(titre1, titre2, id)


8950 paires positives
95882 paires negatives


Création du modèle

In [ ]:
def similarite_levenshtein(chaine1, chaine2):
    return 1 - edit_distance(chaine1,chaine2) / max(len(chaine1),len(chaine2))

def similaires(chaine1, chaine2):
    return similarite_levenshtein(chaine1.upper(), chaine2.upper()) > 0.66 or \
            chaine1.upper().startswith(chaine2.upper()) or \
            chaine2.upper().startswith(chaine1.upper())            
            
def tp(preds, targets):
    tp = len([1 for i in range(len(preds)) if preds[i] == 1 and targets[i] == 1])
    return tp

def fp(preds, targets):
    fp = len([1 for i in range(len(preds)) if preds[i] == 1 and targets[i] == 0])
    return fp

def tn(preds, targets):
    tn = len([1 for i in range(len(preds)) if preds[i] == 0 and targets[i] == 0])
    return tn

def fn(preds, targets):
    fn = len([1 for i in range(len(preds)) if preds[i] == 0 and targets[i] == 1])
    return fn

def accuracy(tp_, tn_, fp_, fn_):
    return (tp_ + tn_)/ (tp_ + tn_ + fp_ + fn_)
    
def accuracy_negative(tp_, tn_, fp_, fn_):
    return (tp_ + tn_)/ (tp_ + tn_ + fp_ + fn_)

def precision(tp_, fp_):    
    return tp_ / (tp_ + fp_)

def precision_negative(tn_, fn_):    
    return tn_ / (tn_ + fn_)

def recall(tp_, fn_):
    return tp_ / (tp_ + fn_)

def recall_negative(tn_, fp_):
    return tn_ / (tn_ + fp_)

def f1_score(p, r):
    return 2 * p * r / (p + r)


In [ ]:
def test(data):
  preds = []
  for i in range(0, len(data[1])):
    preds.append(int(similaires(data[0][i], data[1][i]) == data[2][i]))
  print(len(preds))
  print(data[2])
  print(preds)
   
    # preds = [1 if preds[i][0] > 0.1 else 0 for i in range(preds.shape[0])]
  tp_ = tp(preds, data[2])
  fp_ = fp(preds, data[2])
  fn_ = fn(preds, data[2])
  tn_ = tn(preds, data[2])
  a = accuracy(tp_, tn_, fp_, fn_)
  p = precision(tp_, fp_)
  r = recall(tp_, fn_)
  f1 = f1_score(p, r)

  p_n = precision_negative(tn_, fn_)
  r_n = recall_negative(tn_, fp_)
  f1_n = f1_score(p_n, r_n)

  print(f"tp: {tp_}, nb_pos: {len([1 for y in data[2] if y == 1])}")
  print(f"tn: {tn_}, nb_neg: {len([1 for y in data[2] if y == 0])}")
  print(f"fp: {fp_}, nb_pos: {len([1 for y in data[2] if y == 1])}")
  print(f"fn: {fn_}, nb_neg: {len([1 for y in data[2] if y == 0])}")
  print(f"accuracy: {a} ")
  print('positives')
  print(f"precision: {p}, recall: {r}, f1: {f1}")
  print('negatives')
  print(f"precision: {p_n}, recall: {r_n}, f1: {f1_n}")

test(train_data)

16500
[0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 